In [50]:
x = 3 #Gyperparameter

In [187]:
disk = {}
main_memory = {}
transactions = {}

i = 0
transaction_number = -1
with open("./Sample Test Cases/Part 1/input.txt") as inp :
    for line in inp :
        if i == 0 :
            db_vals = str(line)
            print(db_vals, end = " ")
            db_vals =db_vals[ : -1 ]
            db_vals = db_vals.split(" ")
            print( db_vals )
            j = 0
            while j < len(db_vals) :
                if len( db_vals ) <= j + 1 :
                    break
                db_vals[j] = db_vals[j].replace("\n", "")
                print( db_vals[j], " = ", end = " " )
                db_vals[j + 1] = db_vals[j + 1].replace("\n", "")
                print( db_vals[j + 1] )
                db_vals[j] = db_vals[j].strip()
                disk[db_vals[j]] = int(db_vals[j + 1])
                main_memory[db_vals[j]] = None
                j += 2
            i += 1
        else :
            cmd = line[:-1]
            if cmd == "" :
                transaction_number += 1
                transactions[transaction_number] = []
                print("New transaction ", transaction_number )
            else :
                transactions[transaction_number].append(cmd)
            
                

A 8 B 8 C 5
 ['A', '8', 'B', '8', 'C', '5']
A  =  8
B  =  8
C  =  5
New transaction  0
New transaction  1
New transaction  2


In [188]:
def main_disk_helper() :
    out = ""
    in_main = False
    for item in list(main_memory.keys()) :
        if main_memory[item] != None :
            in_main = True
            out += ( str(item) + " " )
            out += ( str(main_memory[item]) + " " )
    if in_main :
        while out[-1] == " " :
            out = out[ :-1]
    out += "\n"
    for item in list(disk.keys()) :
        if disk[item] != 0 :
            out += ( str(item) + " " )
            out += ( str(disk[item]) + " " )
    while out[-1] == " " :
        out = out[ :-1]
        
    out += "\n"
    return out

def read( item ) :
    if main_memory[item] != None : #Checking if it was already red, if yes then return that value only
        return [main_memory[ item ], -1]
    else : #If it was not red, then read from disk and update main_memory
        temp = disk[item]
        main_memory[item] = temp
        return [temp, -1 ]
    
def write( trans, item, value ) :
    out = "<" + trans + ", " + item + ", " + str(main_memory[item]) + ">\n"
    main_memory[item] = value
    out += main_disk_helper()
    return [-1, out]
    
def output( item ) :
    disk[item] = main_memory[item]
    return [-1, -1]
    
def start( trans ) :
    out = "<START " + str(trans) + ">\n"
    out += main_disk_helper()
    return [-1, out]

def commit( trans, items_used ) :
    out = "<COMMIT " + str(trans) + ">\n"
    for item in list(main_memory.keys()) :
        if item in items_used :
            if main_memory[item] != None :
                if trans == "T1" :
                    print(item, "Main memory = ", main_memory[item], "Disk = ", disk[item] )
                disk[item] = main_memory[item]
                if trans == "T1" :
                    print("After update")
                    print(item, "Main memory = ", main_memory[item], "Disk = ", disk[item] )
    out += main_disk_helper()
    return [-1, out]

In [189]:
trans_name = {}
values = {}
items_trans = {}
out = open("output.txt", "w")
while len(transactions) > 0 :
    for trans in list(transactions.keys()) :
#         print(trans)
        if trans not in list(items_trans.keys()) :
            items_trans[trans] = []
        i = 0
        while i < x :
            ret = [-1, -1]
            cmd = transactions[trans][0]
            print(cmd)
            transactions[trans].remove(cmd)
            cmd = cmd.split(" ")
            if cmd[0][0] == "T" :
#                 print(trans)
                trans_name[trans] = cmd[0]
                ret = start( trans_name[trans] )
                i -= 1
            elif cmd[0][0:4] == "READ" :
                cmd = cmd[0].split("(")
                cmd = cmd[1].split(")")
                item, val = cmd[0].split(",")
                if item not in items_trans[trans] :
                    items_trans[trans].append(item)
                ret = read(item)
                values[val] = ret[0]
            elif cmd[0][0:5] =="WRITE" :
                cmd = cmd[0].split("(")
                cmd = cmd[1].split(")")
                item, val = cmd[0].split(",")
                if item not in items_trans[trans] :
                    items_trans[trans].append(item)
                ret = write( trans_name[trans], item, values[val] )
            elif cmd[0][0:6] == "OUTPUT" :
                cmd = cmd[0].split("(")
                item = cmd[1].split(")")[0]
                ret = output( item )
            else :
                lhs = cmd[0].strip()
                if lhs not in list(values.keys()) :
                    print(lhs, " is not in values keys")
                cmd = cmd[2]
                print(cmd)
                if "*" in cmd :

                    val, num = cmd.split("*")
                    num = int(num)
                    values[lhs] = values[val] * num
#                         print("Multiply", values[lhs])
                elif "+" in cmd :
#                         print("Add")
                    val, num = cmd.split("+")
                    num = int(num)
                    values[lhs] = values[val] + num
#                         print("Add", values[lhs])
                elif "-" in cmd :
#                         print("Substract")
                    val, num = cmd.split("-")
                    num = int(num)
                    values[lhs] = values[val] - num
#                         print("Substract", values[lhs])
                elif "/" in cmd :
#                         print("Divide")
                    val, num = cmd.split("/")
                    num = int(num)
                    values[lhs] = values[val] / num
#                         print("Divide", values[lhs])
            i += 1
            if len(transactions[trans]) == 0 :
                ret = commit( trans_name[trans], items_trans[trans] )
                transactions.pop(trans)
                i = x

            if ret[1] != -1 :
                print( ret[1] )
                out.write(ret[1])

                
out.close()
        
        

T1 8
<START T1>

A 8 B 8 C 5

READ(A,t)
t := t*2
t*2
WRITE(A,t)
<T1, A, 8>
A 16
A 8 B 8 C 5

T2 9
<START T2>
A 16
A 8 B 8 C 5

READ(C,t1)
READ(A,t2)
t1 := t1+2
t1+2
T3 8
<START T3>
A 16 C 5
A 8 B 8 C 5

READ(B,t)
t := t+1
t+1
WRITE(C,t)
<T3, C, 5>
A 16 B 8 C 9
A 8 B 8 C 5

READ(B,t)
t := t*2
t*2
WRITE(B,t)
<T1, B, 8>
A 16 B 16 C 9
A 8 B 8 C 5

WRITE(C,t1)
<T2, C, 9>
A 16 B 16 C 7
A 8 B 8 C 5

t1 := t1-2
t1-2
t1 := t1+2
t1+2
READ(C,t)
t := t+1
t+1
WRITE(B,t)
<T3, B, 16>
A 16 B 8 C 7
A 8 B 8 C 5

OUTPUT(A)
OUTPUT(B)
A Main memory =  16 Disk =  16
After update
A Main memory =  16 Disk =  16
B Main memory =  8 Disk =  8
After update
B Main memory =  8 Disk =  8
<COMMIT T1>
A 16 B 8 C 7
A 16 B 8 C 5

WRITE(A,t1)
<T2, A, 16>
A 7 B 8 C 7
A 16 B 8 C 5

OUTPUT(C)
OUTPUT(A)
<COMMIT T2>
A 7 B 8 C 7
A 7 B 8 C 7

OUTPUT(C)
OUTPUT(B)
<COMMIT T3>
A 7 B 8 C 7
A 7 B 8 C 7

